In [1]:
import os
os.chdir('/home/nmduy/CA683/Credit Project')
#from fancyimpute import IterativeImputer as MICE
import pandas as pd
import numpy as np
import statsmodels.formula.api as sm
import random

random.seed(1509)

In [2]:
data = pd.read_csv('data/encoded_train_vif_dropna.csv')
data_remove_id = data.drop(['SK_ID_CURR'], axis=1)

##### REMOVE 1-VALUE FT #####
remove_ft = []
for col in list(data_remove_id.drop('TARGET', axis=1).columns):
    if data_remove_id[col].sum() == 0 or data_remove_id[col].sum() == len(data_remove_id[col]):
        remove_ft += [col]
len(remove_ft)

5

In [3]:
data.head()

,NAME_CONTRACT_TYPE,CODE_GENDER,AMT_INCOME_TOTAL,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,CNT_FAM_MEMBERS,...,ORGANIZATION_TYPE_Security Ministries,ORGANIZATION_TYPE_Self-employed,ORGANIZATION_TYPE_Services,ORGANIZATION_TYPE_Telecom,ORGANIZATION_TYPE_Trade,ORGANIZATION_TYPE_Transport,ORGANIZATION_TYPE_University,ORGANIZATION_TYPE_XNA,SK_ID_CURR,TARGET
0,1,1.0,202500.0,351000.0,0.018801,637.0,3648.0,2120,-1.0,1.0,...,0,0,0,0,0,0,0,0,100002,1
1,0,1.0,67500.0,135000.0,0.010032,225.0,4260.0,2531,26.0,1.0,...,0,0,0,0,0,0,0,0,100004,0
2,1,1.0,99000.0,454500.0,0.035792,1588.0,4970.0,477,-1.0,2.0,...,0,0,0,0,0,0,0,0,100008,0
3,1,0.0,171000.0,1395000.0,0.035792,3130.0,1213.0,619,17.0,3.0,...,0,0,0,0,0,0,0,0,100009,0
4,1,1.0,360000.0,1530000.0,0.003122,449.0,4597.0,2379,8.0,2.0,...,0,0,0,0,0,0,0,0,100010,0


In [29]:
##### MAKE IT BALANCE #####
numb_pos_samples = data_remove_id['TARGET'].sum()
numb_neg_samples = len(data_remove_id) - numb_pos_samples
portion_pos_train = 0.8
numb_pos_train = int(numb_pos_samples * portion_pos_train)
numb_pos_test = numb_pos_samples - numb_pos_train

index_pos = data_remove_id.index[data_remove_id['TARGET'] == 1].tolist()
index_neg = [i for i in range(len(data_remove_id)) if i not in index_pos]

data_pos = data_remove_id.loc[index_pos, :]
data_neg = data_remove_id.loc[index_neg, :]

# shuffle
data_pos = data_pos.sample(frac=1)
data_neg = data_neg.sample(frac=1)

data_pos_train = data_pos.iloc[0:numb_pos_train,:]
data_pos_test = data_pos.iloc[numb_pos_train:, :]

data_neg_train = data_neg.iloc[0:2*numb_pos_train, :]
data_neg_test = data_neg.iloc[2*numb_pos_train:, :]

data_balance = pd.concat([data_pos_train, data_neg_train], ignore_index=True)
data_remain = pd.concat([data_pos_test, data_neg_test], ignore_index=True)
data_balance = data_balance.drop(remove_ft, axis=1)
data_remain = data_remain.drop(remove_ft, axis=1)
print(f"Numb train/test samples: {len(data_balance)} / {len(data_remain)}")

Numb train/test samples: 40308 / 160776


In [30]:
##### FIND CORRELATION MATRIX #####
corr_matrix = data_balance.corr()
corr_matrix = corr_matrix.replace(1, 0)

In [31]:
corr_max = corr_matrix.max(axis = 0)
corr_min = corr_matrix.min(axis = 0)
corr_max_idx = corr_matrix.idxmax(axis = 0)
corr_min_idx = corr_matrix.idxmin(axis = 0)

In [32]:
dict_corr_max = dict(corr_max)
dict_corr_max_idx = dict(corr_max_idx)
dict_corr_max_filter = {}
for x in dict_corr_max.keys():
    if dict_corr_max[x] > 0.8:
        dict_corr_max_filter[x] = dict_corr_max[x]
dict_corr_max_idx_filter = {}
for x in dict_corr_max_filter.keys():
   dict_corr_max_idx_filter[x] = dict_corr_max_idx[x]

In [33]:
dict_corr_max_filter

{}

In [34]:
dict_corr_max_idx_filter

{}

In [35]:
dict_corr_min = dict(corr_min)
dict_corr_min_idx = dict(corr_min_idx)
dict_corr_min_filter = {}
for x in dict_corr_min.keys():
    if dict_corr_min[x] < -0.8:
        dict_corr_min_filter[x] = dict_corr_min[x]
dict_corr_min_idx_filter = {}
for x in dict_corr_min_filter.keys():
   dict_corr_min_idx_filter[x] = dict_corr_min_idx[x]

In [36]:
dict_corr_min_filter

{}

In [37]:
dict_corr_min_idx_filter

{}

In [38]:
##### REMOVE COLLINEAR ####
# remove_ft = ['AMT_GOODS_PRICE', 'REGION_RATING_CLIENT_W_CITY', 'OBS_60_CNT_SOCIAL_CIRCLE', 'CNT_CHILDREN', 
#              'LIVE_REGION_NOT_WORK_REGION', 'NAME_INCOME_TYPE_Working', 'NAME_EDUCATION_TYPE_Secondary / secondary special']
# remove_ft = ['LIVE_REGION_NOT_WORK_REGION', 'DEF_60_CNT_SOCIAL_CIRCLE']
# remove_ft = ['AMT_GOODS_PRICE', 'REGION_RATING_CLIENT_W_CITY', 'OBS_60_CNT_SOCIAL_CIRCLE']
remove_ft = []
if len(remove_ft) > 0:
    data_balance_rm = data_balance.drop(remove_ft, axis=1)
    data_remain_rm = data_remain.drop(remove_ft, axis=1)
else:
    data_balance_rm = data_balance.copy()
    data_remain_rm = data_remain.copy()

data_balance_rm.shape

(40308, 109)

In [39]:
categorical_name = ['NAME_CONTRACT_TYPE', 'CODE_GENDER']
pc_pos = {}
for col in list(data_balance_rm.columns):
    if 'NAME' in col or 'OCCUPATION' in col or 'WEEKDAY' in col or 'ORGANIZATION' in col:
        categorical_name += [col]
    if col in categorical_name:
        values = np.asarray(list(data_remove_id[col]))
        pc_pos[col] = np.sum(values) / len(values)
remove_ft_dominant = []
for col in categorical_name:
    if pc_pos[col] < 0.01 or pc_pos[col] > 0.99:
        remove_ft_dominant += [col]

In [40]:
len(remove_ft_dominant)

37

In [41]:
##### REMOVE DOMINANT FT IN BINARY #####
data_balance_rm = data_balance_rm.drop(remove_ft_dominant, axis=1)
data_remain_rm = data_remain_rm.drop(remove_ft_dominant, axis=1)
data_balance_rm.shape

(40308, 72)

In [42]:
##### RUN LOGISTIC REGRESSION #####
new_column_names = ["X"+str(x) for x in range(len(list(data_balance_rm.columns))-1)]
new_column_names += ['Y']
data_balance_rm.columns = new_column_names
data_remain_rm.columns = new_column_names

In [45]:
all_columns = "+".join(data_balance_rm.columns.difference(["Y"]))
my_formula = "Y~" + all_columns
result = sm.logit(formula=my_formula, data=data_balance_rm).fit(method='nm', maxiter=500)
print(result.summary())

                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                40308
Model:                          Logit   Df Residuals:                    40237
Method:                           MLE   Df Model:                           70
Date:                Thu, 16 Apr 2020   Pseudo R-squ.:                 0.02587
Time:                        17:46:02   Log-Likelihood:                -24993.
converged:                      False   LL-Null:                       -25657.
Covariance Type:            nonrobust   LLR p-value:                1.557e-231
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept   -2.982e-06        nan        nan        nan         nan         nan
X25[T.True]  1.407e-05        nan        nan        nan         nan         nan
X0          -1.778e-06        nan        nan    

In [25]:
dict_p = dict(result.pvalues)
keep_ft = []
for key, val in dict_p.items():
    if val < 0.999 and key != 'Intercept':
        keep_ft += [key]
len(keep_ft)

41

In [26]:
all_columns = "+".join(keep_ft)
my_formula = "Y~" + all_columns
result = sm.logit(formula=my_formula, data=data_balance_rm).fit(method='nm', maxiter=2500)
print(result.summary())

Optimization terminated successfully.
         Current function value: 0.684610
         Iterations: 962
         Function evaluations: 1251
                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:                26872
Model:                          Logit   Df Residuals:                    26830
Method:                           MLE   Df Model:                           41
Date:                Thu, 16 Apr 2020   Pseudo R-squ.:                 0.01232
Time:                        16:48:41   Log-Likelihood:                -18397.
converged:                       True   LL-Null:                       -18626.
Covariance Type:            nonrobust   LLR p-value:                 5.090e-72
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -2.343e-05      0.064     -0.000      1.000      -0.126   

In [47]:
##### EVALUATE #####
def logPredict(modelParams, X):  
    probabilities = modelParams.predict(X)
    classify = [1 if x >= 0.5 else 0 for x in probabilities]
    return probabilities, classify
    
def calculate_metric(gtnp, pdnp):
    # input are numpy vector
    total_samples = len(gtnp)
    #print(f"Total sample: {total_samples}")
    total_correct = np.sum(gtnp == pdnp)
    accuracy = total_correct / total_samples
    gt_pos = np.where(gtnp == 1)[0]
    gt_neg = np.where(gtnp == 0)[0]
    TP = np.sum(pdnp[gt_pos])
    TN = np.sum(1 - pdnp[gt_neg])
    FP = np.sum(pdnp[gt_neg])
    FN = np.sum(1 - pdnp[gt_pos])
    precision = TP / (TP+FP)
    recall = TP/(TP+FN)
    f1 = 2*precision*recall/(precision+recall)
    metrics = {}
    metrics['accuracy'] = str(accuracy)
    metrics['precision'] = str(precision)
    metrics['recall'] = str(recall)
    metrics['f1'] = str(f1)
    metrics['tp'] = str(int(TP))
    metrics['tn'] = str(int(TN))
    metrics['fp'] = str(int(FP))
    metrics['fn'] = str(int(FN))

    return metrics


In [48]:
_,prediction = logPredict(result, data_remain_rm.drop(['Y'], axis=1))
pred_np = np.asarray(prediction)
lbl_np = np.asarray(data_remain_rm['Y'])
metric = calculate_metric(lbl_np, pred_np)
metric

{'accuracy': '0.9791075782455093',
 'precision': 'nan',
 'recall': '0.0',
 'f1': 'nan',
 'tp': '0',
 'tn': '157417',
 'fp': '0',
 'fn': '3359'}

In [49]:
_,prediction = logPredict(result, data_balance_rm.drop(['Y'], axis=1))
pred_np = np.asarray(prediction)
lbl_np = np.asarray(data_balance_rm['Y'])
metric = calculate_metric(lbl_np, pred_np)
metric

{'accuracy': '0.6666914756375906',
 'precision': '1.0',
 'recall': '7.442691277165823e-05',
 'f1': '0.00014884274763712137',
 'tp': '1',
 'tn': '26872',
 'fp': '0',
 'fn': '13435'}